In [2]:
import os
import json
from ipytree import Node, Tree
from traitlets import Unicode
from ipylab import JupyterFrontEnd, Panel
from ipywidgets import Button, Layout, HBox, VBox
# from fnmatch import fnmatch
# from pathlib import PurePath

In [3]:
def dexiedict(db):
    d = {}
    for i in range(len(db)):
        repo = db[i]['repoid']
        namedict = {db[i]['name']:{}}
        if repo in d:
            d[repo].update(namedict)
        else:
            d[repo] = namedict
    return d

# print(dexiedict(dexie))

{'https://github.com/dg-data/binder-env': {'app-global.ipynb': {}, 'dill.ipynb': {}, 'scrape (3).ipynb': {}, 'scrape.ipynb': {}, 'thread.ipynb': {}}, 'https://github.com/dg-data/dockernels': {'binderupload.ipynb': {}, 'dill.ipynb': {}}, 'https://github.com/dg-data/ner-env': {'NERmap.ipynb': {}}, 'https://github.com/jupyterlab/jupyterlab-plugin-playground': {'Untitled.ipynb': {}}, 'https://github.com/likethebourbon/editing-workflow': {'editing-workflow.ipynb': {}}, 'https://github.com/psychemedia/selenium-demoscraper': {'Firefox demo.ipynb': {}, 'Untitled.ipynb': {}}, 'https://github.com/rcmlz/edu-binder-env': {'pygame.ipynb': {}}}


In [4]:
class TreeNode(Node):
    repo = Unicode("").tag(sync=True)

In [5]:
def create_tree_widget(root, path, depth=0):
    node = Tree() if depth == 0 else TreeNode()
    for name, children in root.items():
        if len(children) == 0:
            leaf = TreeNode(name)
            leaf.repo = path
            leaf.icon = 'file'
            leaf.icon_style = 'warning'
            node.add_node(leaf)
        else:
            subtree = create_tree_widget(children, name, depth + 1)
            subtree.icon = 'folder'
            subtree.icon_style = 'info'
            subtree.name = name
            node.add_node(subtree)
    return node

In [6]:
dexie = json.load(open('browser.json',))
file_tree = create_tree_widget(dexiedict(dexie), '')
# file_tree

In [20]:
app = JupyterFrontEnd()
open_button = Button(description='Open', button_style='success', icon='folder')
expand_button = Button(description='Expand', button_style='info', icon='chevron-down')
collapse_button = Button(description='Collapse', button_style='info', icon='chevron-right')
buttonbox = HBox([open_button, expand_button, collapse_button], layout=Layout(overflow='unset'))


In [10]:
def expand_tree(tree, expand=True):
    for node in tree.nodes:
        node.opened = expand

def on_expand_click(b):
    expand_tree(file_tree)
        
def on_collapse_click(b):
    expand_tree(file_tree, False)

expand_button.on_click(on_expand_click)
collapse_button.on_click(on_collapse_click)

In [19]:
def on_open_clicked(b):
    for node in file_tree.selected_nodes:
        for row in dexie:
            # ['repoid']
            repo = row.split('id:')[1].split(' ')[0]
            # ['name']
            if repo == node.repo and node.name == row.split(' path:')[1]:
                app.commands.execute('my:load-from-browser', {'pk': row})

open_button.on_click(on_open_clicked)

In [12]:
panel = Panel(children=[buttonbox, file_tree])
panel.title.label = 'IndexedDB Browser'
panel.title.icon_class = 'jp-FileIcon'
panel.layout = Layout(overflow='auto')
expand_tree(file_tree, True)
file_tree.layout = Layout(overflow='auto')

In [14]:
app.shell.add(panel, 'left', {'rank': 10000})

In [23]:
"""
def collect_files(root_path='..'):
    files = []
    for dirpath, dirnames, filenames in os.walk(root_path, followlinks=True):
        dirnames[:] = [d for d in dirnames] # if d not in EXCLUDES]
        for f in filenames:
            fullpath = PurePath(dirpath).relative_to(root_path).joinpath(f)
            if fullpath.parts not in files:
                files.append(fullpath.parts)
    files.sort()
    return files

files = collect_files()
tree = {}
for f in files:
    node = tree
    for part in f:
        if part not in node:
            node[part] = {}
        node = node[part]
files
"""